# create database

> Core database functionality for the keybindings_fps app. This module is meant to run once to create the database and tables. Don't run this module again if the database already exists, because it will drop the existing tables.

In [2]:
#| default_exp create_db

In [3]:
#| hide
from nbdev.showdoc import *

In [4]:
#| export
from pathlib import Path
from fastcore.test import *
from fasthtml.common import *

In [5]:
#| export
def get_project_root():
    """Get the project root directory from either notebook or module context"""
    try:
        # When running as a module
        return Path(__file__).parent.parent.parent
    except NameError:
        # When running in notebook
        return Path.cwd().parent

In [6]:
#| hide
get_project_root()

Path('/home/jelle/code/keybindings_fps')

In [7]:
#|export
def init_db(data_dir=None):
    """Initialize the database connection
    Args:
        data_dir: Optional path to data directory. If None, uses project's data dir
    """
    if data_dir is None:
        data_dir = get_project_root() / 'data'
    data_dir.mkdir(exist_ok=True)
    return database(data_dir / 'game_bindings.db')

In [9]:
db = init_db()

In [ ]:
#|export
def create_tables(db, overwrite_existing=False):
    """Create all required database tables.
    WARNING: This will drop existing tables!
    """
    if overwrite_existing:
        replace=True
        transform=False
    else:
        replace=False
        transform=True
    
    tables = ['categories', 'actions', 'games', 'game_keys', 'modifiers', 'bindings']
    
    # Drop tables if they exist
    for table in tables:
        if table in db.t:
            db.t[table].drop()
    
    # Create categories table
    db.t.categories.create(
        id=int,
        name=str,
        description=str,
        pk='id',
        not_null=['name'],
        transform=transform
        replace=replace
    )
    
    # Create actions table
    db.t.actions.create(
        id=int,
        name=str,
        description=str,
        category_id=int,
        pk='id',
        not_null=['name', 'category_id'],
        transform=transform
        replace=replace
    )
    
    # Create games table
    db.t.games.create(
        id=int,
        name=str,
        game_type=str,
        image=bytes,
        pk='id',
        not_null=['name'],
        transform=transform
        replace=replace
    )
    
    # Create game_keys table
    db.t.game_keys.create(
        id=int,
        name=str,
        pk='id',
        not_null=['name'],
        transform=transform
        replace=replace
    )
    
    # Create modifiers table
    db.t.modifiers.create(
        id=int,
        name=str,
        pk='id',
        not_null=['name'],
        transform=transform
        replace=replace
    )
    
    # Create bindings table
    db.t.bindings.create(
        id=int,
        game_id=int,
        action_id=int,
        key_id=int,
        modifier_id=int,
        description=str,
        sort_order=int,
        pk='id',
        not_null=['game_id', 'action_id', 'key_id', 'modifier_id'],
        transform=transform
        replace=replace
    )


### Open existing database in project data directory

In [8]:
db = init_db()

## Modify structure of database tables

#### Add a new column to an existing table

In [9]:
#| export
def add_clmn_to_table(db, table, column, col_type, **kwargs):
    """Add a new column to an existing table"""
    if column in db.t[table].c:
        print(f"Column {column} already exists in table {table}")
    else:
        return db.t[table].add_column(column, col_type, **kwargs)

In [67]:
db.t['bindings'].c

action_id, game_id, id, key_id, modifier_id, sort_order

In [58]:
add_clmn_to_table(db, 'bindings', 'test', str, not_null_default='test text')

<Table bindings (id, game_id, action_id, key_id, modifier_id, sort_order, test)>

In [59]:
test_eq(db.t['bindings'].columns[-1].name, 'test')

#### Delete/drop a column from an existing table

In [10]:
#| export
def drop_clmn_from_table(db, table, column):
    """Drop a column from an existing table"""
    if column not in db.t[table].c:
        print(f"Column {column} does not exist in table {table}")
    else:
        return db.t[table].drop_column(column)

In [62]:
drop_clmn_from_table(db, 'bindings', 'test')

<Table bindings (id, game_id, action_id, key_id, modifier_id, sort_order)>

In [66]:
test_eq('test' not in db.t['bindings'].c, True)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()